In [5]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
from google.colab import userdata
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


In [7]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

class ConversationAssistant:
    def __init__(self, llm_model) -> None:
        self.quant = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant = True,
            bnb_4bit_compute_dtype=torch.float16,
        )
        self.model = AutoModelForCausalLM.from_pretrained(llm_model, quantization_config=self.quant)
        self.tokenizer = AutoTokenizer.from_pretrained(llm_model)

    def generate_response(self, conversation):
        model_inputs = self.tokenizer.apply_chat_template(conversation, return_tensors="pt").to("cuda")
        generated_ids = self.model.generate(model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]


In [8]:
    conversation = [
        {"role": "user", "content": "which is your favourite movie?"},
        {"role": "assistant", "content": "Well, I like twilight as it is a fictional movie and i find its character quite interesting"},
        {"role": "user", "content": "Do you have any other choice?"}
    ]

    llm_model = "mistralai/Mistral-7B-Instruct-v0.2"
    assistant = ConversationAssistant(llm_model)

    response = assistant.generate_response(conversation)
    print(response)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] which is your favourite movie? [/INST]Well, I like twilight as it is a fictional movie and i find its character quite interesting</s> [INST] Do you have any other choice? [/INST] Yes, I have other preferences as well. I also enjoy science fiction movies like Interstellar and The Matrix. And I must admit, I have a soft spot for comedies like Anchorman and Superbad. But if we're talking about all-time favorites, Twilight will always have a special place in my "heart" (pun intended!) but please keep in mind that I'm just a computer program and don't have the ability to have feelings or
